In [1]:
import re
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import gc

In [ ]:
datos=pd.read_csv("dataset_dialogos.csv")
datos=datos.loc[:len(datos)//10]["dialogue"]

In [ ]:
datos_preprocesado=datos[:100]
datos_preprocesado

In [ ]:
#Prueba del tokenizer y embeddings model

In [2]:
from transformers import BertTokenizer,BertModel
model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
texto = "pablo cómo te fue ayer en la entrevista"
tokens = tokenizer.tokenize(texto)
print(tokens)
# ['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
tokens_id=tokenizer.convert_tokens_to_ids(tokens)
#[169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
embedding_vector = model.embeddings.word_embeddings
embedding_vector 
#embedding_vector.shape

['pa', '##blo', 'cómo', 'te', 'fue', 'ay', '##er', 'en', 'la', 'entrevista']


Embedding(119547, 768, padding_idx=0)

In [3]:
#Funciones de etiquetado
from typing import List, Union
from transformers import BertTokenizer,BertModel

model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def separar_palabras(texto:str)->List[str]:
    primera_letra=0
    palabras=[]
    
    #palabras que terminan por espacio
    for i in range(len(texto)):
        if texto[i] == " ":
            palabra=texto[primera_letra:i]
            primera_letra=i+1
            palabras.append(palabra)
        #palabras que terminan por cambio de parrafo
        if texto[i]=="\n":
            palabra=texto[primera_letra:i]
            primera_letra=i+1  
            palabras.append(palabra)

        if texto[i]=="¡" or texto[i]=='"' or texto[i]=="'":
            primera_letra+=1
    
    #agrego ultima palabra
    palabras.append(texto[primera_letra:])
    return palabras

def clasificar_mayusculas(palabra:str)->int:
    """Toma una palabra y cuenta sus mayusculas"""
    cantidad=0
    
    for i in range(len(palabra)):
        if palabra[i].lower()!=palabra[i]:
            cantidad+=1
            
    if cantidad==len(palabra): #todas son mayuscula
        res=3
    elif cantidad==1 and palabra[0].lower()!=palabra[0]: #solo la primera es mayuscula
        res=1
    elif cantidad>0: #alguna es mayuscula (iPhone tiene que estar aca)
        res=2
    else: #Ninguna de las anteriores
        res=0
    return res
    
def asignar_labels(datos: pd.Series, path=None)->pd.DataFrame:
    punt_inicial=[]
    instancias=[]
    token_ids=[]
    tokens=[]
    capitalizacion=[]
    pfinal=[]
    instancia=0
    print("Comenzando procesamiento")
    for dialogo in datos:
        if instancia % (len(datos)//10)==0:
            print(f"procesando un {instancia/len(datos)*100}%")
            
        palabras_originales= separar_palabras(dialogo)
        palabras_lower= [palabra.lower() for palabra in palabras_originales]
        instancia+=1
        #print(palabras_lower)

        punt_inicial_seguridad=[]
        instancias_seguridad=[]
        capitalizacion_seguridad=[]
        palabra_tokenizada_ids_seguridad=[]
        tokens_seguridad=[]
        pfinal_seguridad=[]
        es_seguro=True
        # hola, ¿Como estas. ? ### hola, ; ¿Como ; estas?
        #token  hola ,["¿", como] estas ?
        for idx, palabra in enumerate(palabras_lower):
            
            hay_signo_apertura=False 
            palabra_tokenizada= tokenizer.tokenize(palabra)

            #Recordartorio SON PALABRAS. ejemplos: '¿Que' o "escuela?" o "Franquito," o "Emi."  "¿ que ?"
            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[0]=="¿":
                    palabra_tokenizada=palabra_tokenizada[1:]
                    hay_signo_apertura=True
                    punt_inicial_seguridad.append(1) # ¿
                
             
            if len(palabra_tokenizada)>0:
                #estas ?
                if palabra_tokenizada[-1]=="?":        
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    
                    #relleno las demas listas
                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(1) #?
                    
                elif palabra_tokenizada[-1]==".":
                    
                    palabra_tokenizada=palabra_tokenizada[:-1] 

                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(2) #.
        
                elif palabra_tokenizada[-1]==",":
                    palabra_tokenizada=palabra_tokenizada[:-1]

                    pfinal_seguridad+=[0]*(len(palabra_tokenizada)-1)
                    pfinal_seguridad.append(3) #,
        
                else:
                    pfinal_seguridad+=[0]*(len(palabra_tokenizada))
                    
            palabra_tokenizada_ids= tokenizer.convert_tokens_to_ids(palabra_tokenizada) 
            
            if hay_signo_apertura:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada)-1)
                
            else:
                punt_inicial_seguridad+=[0]*(len(palabra_tokenizada))
            
            #capitalizacion
            clasificacion=None
            palabra_orginal= palabras_originales[idx]
            if len(palabra_tokenizada)>0:
                if palabra_orginal[0]=="¿" or palabra_orginal[0]=='"' or palabra_orginal[0]=='¡':
                    if palabra_orginal[-1]=='"' or palabra_orginal[-1]=='!' or palabra_orginal[-1]=='?' or palabra_orginal[-1]==',' or palabra_orginal[-1]=='.' or palabra_orginal[-1]=='?':
                        clasificacion= clasificar_mayusculas(palabra_orginal[1:-1])
                    else: 
                         clasificacion= clasificar_mayusculas(palabra_orginal[1:])
                else:   
                    clasificacion= clasificar_mayusculas(palabra_orginal)

            if len(palabra_tokenizada)==0:
                es_seguro=False 
                break
            
            capitalizacion_seguridad+= [clasificacion]*len(palabra_tokenizada)
            palabra_tokenizada_ids_seguridad+=palabra_tokenizada_ids
            tokens_seguridad+=palabra_tokenizada
            instancias_seguridad+=[instancia]*len(palabra_tokenizada)
        
        if es_seguro:
            capitalizacion+=capitalizacion_seguridad
            token_ids+= palabra_tokenizada_ids_seguridad
            tokens+= tokens_seguridad
            instancias+=instancias_seguridad
            punt_inicial+=punt_inicial_seguridad
            pfinal+=pfinal_seguridad
            
            if len(capitalizacion) != len(punt_inicial): #tamaño cantidad de token procesados
                print("dif inicial",instancia,dialogo)
                return
                
            if len(capitalizacion) != len(pfinal):
                print("dif final",instancia,dialogo)
                print(len(capitalizacion))
                print(len(punt_inicial))
                return
                
        else:
            instancia-=1
    
    
        if len(capitalizacion)!=len(punt_inicial):
            print("dif inicial",instancia)
            print(len(capitalizacion))
            print(len(punt_inicial))
            
        if len(capitalizacion)!=len(pfinal):
            print("dif final",instancia)
            print(len(capitalizacion))
            print(len(pfinal))
    
    datos_listos = pd.DataFrame({
    "instancia_id": pd.Series(instancias, dtype="int64"),   # o lo que corresponda
    "token": tokens,
    "token_id": pd.Series(token_ids, dtype="int64"),
    "punt_inicial": pd.Series(punt_inicial, dtype="int8"),
    "capitalizacion": pd.Series(capitalizacion, dtype="int8"),
    "pfinal": pd.Series(pfinal, dtype="int8")
})

    if path!=None:
        datos_listos.to_parquet(path, index=False)
    return datos_listos    
            

#datos_listos=asignar_labels(datos_preprocesado,"datos_modelo.parquet")

In [ ]:
#primer decimo me entra en memoria tarda una hora
datos_listos=asignar_labels(datos,"datos_modelo.parquet") #subtitulso

In [ ]:
#No anda por temas de memoria
def asignar_labels_fraccional(datos: pd.DataFrame, prop: float = 0.1, inicio: int = 0):
    """
    Procesa el dataset en fracciones para evitar problemas de memoria.
    Crea archivos parquet: datos_modelo_0.parquet, datos_modelo_1.parquet, etc.
    """
    # Si datos es dataframe, tomamos SOLO la columna dialogue
    if isinstance(datos, pd.DataFrame):
        if "dialogue" not in datos.columns:
            raise ValueError("El DataFrame no tiene columna 'dialogue'")
        datos = datos["dialogue"]   # <-- ahora siempre es Serie

    n = len(datos)
    tam = int(n * prop)
    total_chunks = (n + tam - 1) // tam  # ceil(n/tam)

    for i in range(inicio, total_chunks):
        print(f"Procesando chunk {i+1}/{total_chunks}...")

        ini = i * tam
        fin = min((i+1)*tam, n)

        chunk = datos.iloc[ini:fin]  # <-- chunk siempre será una Serie

        asignar_labels(
            chunk,
            path=f"datos_modelo_{i}.parquet"
        )
        
        del chunk
        gc.collect()

    print("Procesamiento completo.")


In [4]:
datos=pd.read_csv("dataset_dialogos.csv").reset_index(drop=True)

datos_uno = datos.loc[:len(datos)//10]["dialogue"]
datos_dos = datos.loc[(len(datos)//10) +1 : 2*(len(datos)//10)]["dialogue"]
datos_tres = datos.loc[2*(len(datos)//10) +1 : 3*(len(datos)//10)]["dialogue"]
datos_cuatro = datos.loc[3*(len(datos)//10) +1 : 4*(len(datos)//10)]["dialogue"]
datos_cinco = datos.loc[4*(len(datos)//10) +1 : 5*(len(datos)//10)]["dialogue"]
datos_seis = datos.loc[5*(len(datos)//10) +1 : 6*(len(datos)//10)]["dialogue"]
datos_siete = datos.loc[6*(len(datos)//10) +1 : 7*(len(datos)//10)]["dialogue"]
datos_ocho = datos.loc[7*(len(datos)//10) +1 : 8*(len(datos)//10)]["dialogue"]
datos_nueve = datos.loc[8*(len(datos)//10) +1 : 9*(len(datos)//10)]["dialogue"]
datos_diez = datos.loc[9*(len(datos)//10) +1 : ]["dialogue"]

In [ ]:
datos_listos_uno =asignar_labels(datos_uno,"datos_modelo_uno.parquet")

In [ ]:
datos_listos_dos =asignar_labels(datos_dos,"datos_modelo_dos.parquet")

In [ ]:
datos_listos_tres =asignar_labels(datos_tres,"datos_modelo_tres.parquet")

In [ ]:
datos_listos_cuatro =asignar_labels(datos_cuatro,"datos_modelo_cuatro.parquet")

In [5]:
datos_listos_cinco =asignar_labels(datos_cinco,"datos_modelo_cinco.parquet")

Comenzando procesamiento
procesando un 0.0%
procesando un 9.999964971048572%
procesando un 19.999929942097143%
procesando un 29.999894913145713%
procesando un 39.99985988419429%
procesando un 49.999824855242856%
procesando un 59.999789826291426%
procesando un 69.99975479734%
procesando un 79.99971976838857%
procesando un 89.99968473943714%


In [ ]:
datos_listos_seis =asignar_labels(datos_seis,"datos_modelo_seis.parquet")

In [ ]:
datos_listos_siete =asignar_labels(datos_siete,"datos_modelo_siete.parquet")

In [ ]:
datos_listos_ocho =asignar_labels(datos_ocho,"datos_modelo_ocho.parquet")

In [ ]:
datos_listos_nueve =asignar_labels(datos_nueve,"datos_modelo_nueve.parquet")

In [ ]:
datos_listos_diez =asignar_labels(datos_diez,"datos_modelo_diez.parquet")


In [ ]:
df = pd.read_parquet("datos_modelo_dos.parquet")

In [ ]:
df

In [ ]:
instancias_40000 = df["instancia_id"].unique()[:40000]
datos = df[df["instancia_id"].isin(instancias_40000)]
datos

In [ ]:
datos.to_parquet("datos_eval_RNN.parquet", index=False)